In [2]:
import cv2
import os
import face_recognition

In [3]:
img = cv2.imread(r"C:\Users\Sam\Documents\GitHub\Reconnaissance_Image\individuelle\Samuel Ibghi.jpeg")
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_encoding = face_recognition.face_encodings(rgb_img)[0]

In [4]:
img2 = cv2.imread(r"C:\Users\Sam\Documents\GitHub\Reconnaissance_Image\individuelle\Samuel Ibghi.jpeg")
rgb_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img_encoding2 = face_recognition.face_encodings(rgb_img2)[0]

In [5]:
result = face_recognition.compare_faces([img_encoding], img_encoding2)
print("Result: ", result)

Result:  [True]


In [6]:
import face_recognition
import cv2
import os
import glob
import numpy as np

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load encoding images from path
        :param images_path:
        :return:
        """
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
                        
            if len(face_recognition.face_encodings(img)) > 0:
                face_encoding = face_recognition.face_encodings(img)[0]
            else:
                print(f"No face found in {img}")
                continue
            
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names


In [7]:
# Encode faces from a folder
sfr = SimpleFacerec()
sfr.load_encoding_images(r"C:\Users\Sam\Documents\GitHub\Reconnaissance_Image\individuelle")

3 encoding images found.
Encoding images loaded


In [8]:
# Load Camera
cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()

    # Detect Faces
    face_locations, face_names = sfr.detect_known_faces(frame)
    for face_loc, name in zip(face_locations, face_names):
        
        print(face_loc)
        
        
        
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

        cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

[280 488 452 316]
[136 196 280  52]
[280 488 452 316]
[136 196 280  52]
[280 488 452 316]
[136 196 280  52]
[280 468 452 296]
[136 196 280  52]
[280 468 452 296]
[136 196 280  52]
[280 468 452 296]
[136 212 280  68]
[260 468 432 296]
[136 196 280  52]
[260 468 432 296]
[136 212 280  68]
[280 468 452 296]
[136 212 280  68]
[280 468 452 296]
[120 196 264  52]
[260 488 432 316]
[136 212 280  68]
[260 488 432 316]
[120 212 264  68]
[260 488 432 316]
[120 212 264  68]
[264 484 408 340]
[120 212 264  68]
[260 508 432 336]
[120 212 264  68]
[260 488 432 316]
[120 228 264  84]
